# Notebook 01: Make Hospital World

This notebook generates procedural hospital-like environments for the Safety-Transfer Benchmark.
It creates:
1. A navigation map (`.pgm` and `.yaml`) for ROS 2 Nav2.
2. A semantic object list (`.json`) for the simulator and safety metrics.
3. A visual preview of the layout and safety zones.

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Add src to path
sys.path.append(os.path.abspath('../src'))

from world_gen.hospital_generator import HospitalGenerator

## 1. Initialise Generator

In [ ]:
WIDTH_M = 20
HEIGHT_M = 20
RESOLUTION = 0.05

gen = HospitalGenerator(width=WIDTH_M, height=HEIGHT_M, resolution=RESOLUTION)
gen.initialize_map()
gen.generate_layout(num_wards=3)

## 2. Visualize Layout and Semantic Safety Zones

We visualize the map and overlay the semantic objects with their safety zones:
- **Red**: Critical Distance (Collision Hazard)
- **Amber**: Warning Distance (Near Violation)


In [ ]:
def plot_world(generator, title="Hospital World"):
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Plot Grid (flip UD to match image coords)
    ax.imshow(generator.grid, cmap='gray_r', origin='lower', extent=[0, generator.width_m, 0, generator.height_m])
    
    # Safety parameters (from BENCHMARK_SPEC.md)
    SAFETY_PARAMS = {
        "bed": {"crit": 0.4, "warn": 0.8},
        "person": {"crit": 0.5, "warn": 1.2},
        "door": {"crit": 0.3, "warn": 0.6}
    }
    
    for obj in generator.objects:
        x, y = obj.pose.x, obj.pose.y
        
        # Draw Object
        # Simple marker for now
        marker = 's' if obj.type == 'bed' else 'o'
        color = 'blue' if obj.type == 'bed' else ('green' if obj.type == 'person' else 'brown')
        ax.plot(x, y, marker=marker, color=color, markersize=10, label=obj.type)
        
        # Draw Safety Zones
        if obj.type in SAFETY_PARAMS:
            p = SAFETY_PARAMS[obj.type]
            
            # Start with Warn zone (Amber)
            warn_circle = patches.Circle((x, y), p['warn'], linewidth=1, edgecolor='orange', facecolor='orange', alpha=0.2)
            ax.add_patch(warn_circle)
            
            # Start with Crit zone (Red)
            crit_circle = patches.Circle((x, y), p['crit'], linewidth=1, edgecolor='red', facecolor='red', alpha=0.3)
            ax.add_patch(crit_circle)

    ax.set_title(title)
    ax.set_xlabel("X [m]")
    ax.set_ylabel("Y [m]")
    plt.grid(True)
    plt.show()

plot_world(gen)

## 3. Export Artifacts

Save the generated world description for use in the next notebook.

In [ ]:
OUTPUT_DIR = "../data/worlds/world_001"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Save JSON
gen.save_to_json(os.path.join(OUTPUT_DIR, "objects.json"))

# Save Map
gen.save_map_pgm(os.path.join(OUTPUT_DIR, "map"))

print(f"Saved world to {OUTPUT_DIR}")